In [1]:
import os
import sys
import cv2
import numpy as np
from pandas import read_csv
#pip install moviepy
#pip install requests
from moviepy.editor import *
from matplotlib import pyplot as plt 
import keras
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
import tensorflow as tf


infile="../picture/v.mp4"
outfile="../picture/v-output.mp4"

'''
Code adapted from:
https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/

'''

IMG_SHAPE=[32,32,3]
INPUT_SHAPE=[1,32,32,3]
L2_LAMBDA = 0.0001   
WEIGTHS_PATH="../doc/weights_mltscl_dataaug.hdf5"
OUTPUT_DIM=43
LR = 0.0001
CONF_THRES=0.5

# Labels of circlur signs
cir_cls = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 16,
           17, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]

signnames = read_csv("../doc/signnames.csv").values[:, 1]

def softmax(x):
    """
    Compute softmax values for each sets of scores in x.
    Ref: https://stackoverflow.com/questions/34968722/softmax-function-python
    """
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def build_mltscl():
    """
    Build multiscale CNN. The last layer must be logits instead of softmax.
    Return a compiled Keras model.
    """

    # Regularization
    l2_reg = keras.regularizers.l2(L2_LAMBDA)

    # Build model
    inpt = keras.layers.Input(shape=IMG_SHAPE)
    conv1 = keras.layers.Convolution2D(
        32, (5, 5), padding='same', activation='relu')(inpt)
    drop1 = keras.layers.Dropout(rate=0.1)(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop1)

    conv2 = keras.layers.Convolution2D(
        64, (5, 5), padding='same', activation='relu')(pool1)
    drop2 = keras.layers.Dropout(rate=0.2)(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop2)

    conv3 = keras.layers.Convolution2D(
        128, (5, 5), padding='same', activation='relu')(pool2)
    drop3 = keras.layers.Dropout(rate=0.3)(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop3)

    pool4 = keras.layers.MaxPooling2D(pool_size=(4, 4))(pool1)
    pool5 = keras.layers.MaxPooling2D(pool_size=(2, 2))(pool2)

    flat1 = keras.layers.Flatten()(pool4)
    flat2 = keras.layers.Flatten()(pool5)
    flat3 = keras.layers.Flatten()(pool3)

    merge = keras.layers.Concatenate(axis=-1)([flat1, flat2, flat3])
    dense1 = keras.layers.Dense(1024, activation='relu',
                                kernel_regularizer=l2_reg)(merge)
    drop4 = keras.layers.Dropout(rate=0.5)(dense1)
    output = keras.layers.Dense(
        OUTPUT_DIM, activation='softmax', kernel_regularizer=l2_reg)(drop4)
    model = keras.models.Model(inputs=inpt, outputs=output)

    # Specify optimizer
    adam = keras.optimizers.Adam(
        lr=LR, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model=build_mltscl()
model.load_weights(WEIGTHS_PATH)
#model.summary()

def rgb2gray(image):
    """Convert 3-channel RGB image into grayscale"""
    if image.ndim == 3:
        return (0.299 * image[:, :, 0] + 0.587 * image[:, :, 1] +
                0.114 * image[:, :, 2])
    elif image.ndim == 4:
        return (0.299 * image[:, :, :, 0] + 0.587 * image[:, :, :, 1] +
                0.114 * image[:, :, :, 2])

#识别原型的交通牌
def find_circles(img, mg_ratio=0.4, n_circles=1):
    """
    Find circular objects and return bounding boxes in the format
    [x1, y1, x2, y2]
    """

    targetImg = np.copy(img)
    targetImg = np.uint8(targetImg * 255)
    # Apply Gaussian blur if needed
    n = 13
    targetImg = cv2.GaussianBlur(targetImg, (n, n), 0)

    # Convert to grayscale
    grayImg = np.uint8(rgb2gray(targetImg))
    #grayImg = targetImg
    # param1 is canny threshold, param2 is circle accumulator threshold
    # Set of parameters for GTSDB testing
    # (because of different frame size and recording device)
    # circles = cv2.HoughCircles(grayImg, cv2.HOUGH_GRADIENT, 1, 200,
    #                            param1=60, param2=50, minRadius=5,
    #                            maxRadius=100)
    circles = cv2.HoughCircles(grayImg, cv2.HOUGH_GRADIENT, 1, 200,
                               param1=50, param2=30, minRadius=20,
                               maxRadius=250)

    bboxes = []
    try:
        cir = circles.astype(np.uint16)
        for c in cir[0, :n_circles]:
            r = int(c[2])
            mg = int(r * mg_ratio)
            bboxes.append([c[0] - r - mg, c[1] - r - mg,
                           c[0] + r + mg, c[1] + r + mg])
    except AttributeError:
        pass
    except:
        raise
    return bboxes


def draw_bb(img, bbox, color=(0, 1, 0)):
    """Draw bounding box"""
    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                  color, 2)
    return img

def crop_bb(img, bbox):
    """Crop image by specifying bounding box"""
    bb = np.array(bbox)
    bb = bb * (bb > 0)
    return img[bb[1]:bb[3], bb[0]:bb[2], :]


def demo():
    
    demo_file="../picture/traffic_sign.jpeg"
    #OpenCV读取的格式为BGR 需要转换成RGB进行展现
    demo_img=cv2.imread(demo_file)
    demo_img = cv2.cvtColor(demo_img, cv2.COLOR_BGR2RGB) 
    show_img=demo_img.copy()
    plt.imshow(show_img) 
    plt.show() 
        
    bboxes = find_circles(show_img.copy(), mg_ratio=0.4, n_circles=3)
    for bbox in bboxes:
        crop = crop_bb(show_img, bbox)
        #resized_im = resize(crop)
        resized_im=cv2.resize(crop,(32,32))
        resized_im = np.expand_dims(resized_im, axis=0)
        #label = signnames[predict(model, resized_im)]
        label = np.argmax(model.predict([resized_im]))
        conf = np.max(softmax(model.predict(
            resized_im.reshape(INPUT_SHAPE))[0]))
        # Consider detection only if confidence is larger than threshold
        print("label={} conf={}".format(label,conf))
        if (conf > CONF_THRES ) and  (label in cir_cls) :
            print("Draw label={} conf={}".format(label,conf))
            show_img = draw_bb(show_img, bbox)
            # Put label and confidence
            cv2.putText(show_img, '{}: {:.2f}'.format(signnames[label], conf), (bbox[0], bbox[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 1, 0), 2)
        else:
            print('{}: {:.2f}'.format(label, conf))

    
    plt.imshow(show_img) 
    plt.show() 
demo()


Using TensorFlow backend.


<Figure size 640x480 with 1 Axes>

label=1 conf=0.060786809772253036
1: 0.06
label=23 conf=0.060786809772253036
23: 0.06
label=41 conf=0.060786809772253036
41: 0.06


<Figure size 640x480 with 1 Axes>

In [ ]:
#全局计数器
count=0

#实现回调函数 处理每一帧图像
def invert_VideoFileClip(image):
    
    #仅处理一部分 提高效率
    if count % 10 == 0 :
       
        bboxes = find_circles(image, mg_ratio=0.4, n_circles=3)
        for bbox in bboxes:
            crop = crop_bb(image, bbox)
            #resized_im = resize(crop)
            resized_im=cv2.resize(crop,(32,32))
            resized_im = np.expand_dims(resized_im, axis=0)
            #label = signnames[predict(model, resized_im)]
            logit=softmax(model.predict([resized_im])[0])
            label = np.argmax(logit)
            conf = logit[label]
            # Consider detection only if confidence is larger than threshold
            print("label={} conf={}".format(label,conf))
            if (conf > CONF_THRES ) and  (label in cir_cls) :
                #print("Draw label={} conf={}".format(label,conf))
                img = draw_bb(image, bbox)
                # Put label and confidence
                cv2.putText(image, '{}: {:.2f}'.format(signnames[label], conf), (bbox[0], bbox[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 1, 0), 2)
            else:
                print('{}: {:.2f}'.format(label, conf))
    
    return image

clip = VideoFileClip(infile)
modifiedClip = clip.fl_image(invert_VideoFileClip)
modifiedClip.write_videofile(outfile)

label=39 conf=0.06078679487109184
39: 0.06
label=22 conf=0.060786809772253036
22: 0.06
label=35 conf=0.060786809772253036
35: 0.06
[MoviePy] >>>> Building video ../picture/v-output.mp4
[MoviePy] Writing audio in v-outputTEMP_MPY_wvf_snd.mp3


100%|██████████| 314/314 [00:00<00:00, 349.75it/s]

[MoviePy] Done.
[MoviePy] Writing video ../picture/v-output.mp4



  0%|          | 1/426 [00:00<03:53,  1.82it/s]

label=39 conf=0.06078679487109184
39: 0.06
label=22 conf=0.060786809772253036
22: 0.06
label=35 conf=0.060786809772253036
35: 0.06


...

label=33 conf=0.04921441152691841
33: 0.05
label=41 conf=0.060786809772253036
41: 0.06
label=19 conf=0.060786809772253036
19: 0.06


 44%|████▎     | 186/426 [01:26<01:46,  2.25it/s]

label=41 conf=0.060786809772253036
41: 0.06
label=41 conf=0.060786809772253036
41: 0.06
label=42 conf=0.060786809772253036
42: 0.06


 44%|████▍     | 187/426 [01:27<02:05,  1.91it/s]

label=39 conf=0.060786809772253036
39: 0.06
label=42 conf=0.060786809772253036
42: 0.06
label=41 conf=0.060786809772253036
41: 0.06
